In [1]:
import torch
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np
# label_path = 'data/article/A2_sentence_labeled_four_class_answer.xlsx'
label_path = 'data/article/split_output_A2_1113_answer.xlsx'
only_article_path = 'data/article/A2_二次標註_網路危機訊息標註_1100927_僅全文標註(給致寧).xlsx'
# label_path = 'data/article/partial_A1_article.xlsx'
df = pd.read_excel(label_path)
article_only_df = pd.read_excel(only_article_path)
df = df[['TextID','Title','Crisis_Level']]
print(df.columns)
# print(df)

for idx, val in enumerate(df['TextID']):
    if type(val) != str:
        df['TextID'][idx] = str(val)
stat = [0]*4
for idx, val in enumerate(article_only_df['Crisis_Level']):
    label = 3 - ord(val[0]) + ord('a')
    stat[label] += 1
print(stat)
four_label = df.values
title_list = df['Title'].values
textID_list = df['TextID'].values
# sentence_path = 'data/article/A2_sentence_labeled_four_class.xlsx'
sentence_path = 'data/article/split_output_A2_v2_answer.xlsx'
s_df = pd.read_excel(sentence_path)[[ 'TextID','Title', 'Sentence', '無標註', '自殺與憂鬱', '自殺行為','其他類型']]
for idx, val in enumerate(s_df['TextID']):
    if type(val) != str:
        s_df['TextID'][idx] = str(val)

sentence_label_list = ['無標註', '自殺與憂鬱', '自殺行為', '其他類型']
article_dict = {str(key): {'無標註':'', '自殺與憂鬱':'','自殺行為':'', '其他類型':''} for key in textID_list}
problem_title = []
for index, row in s_df.iterrows():
    article_dict[str(row['TextID'])]['label'] = df[(df['TextID']) == (row['TextID'])]['Crisis_Level']
    flag = 3 - ord(article_dict[str(row['TextID'])]['label'].values[0][0]) + ord('a')
    # if flag == 3:
    #    flag = 1
    # elif flag == 1:
    #    flag = 3
    article_dict[str(row['TextID'])]['Crisis_Level'] = flag
    article_dict[str(row['TextID'])]['label'] = [0.0, 0.0, 0.0, 0.0]
    article_dict[str(row['TextID'])]['label'][flag] += 1.0
    for s_label in sentence_label_list:
      if row[s_label] == 1:
        tar = ' ' + str(row['Sentence'])
        article_dict[str(row['TextID'])][s_label] += tar
        break
all_article = []
for dic in article_dict:
    single_article = pd.DataFrame([article_dict[dic]])
    single_article['TextID'] = dic
    all_article.append(single_article)
all_article_df = pd.concat(all_article,axis=0, ignore_index=True)
print(all_article_df.columns)
all_article_df['TextID'] = all_article_df['TextID'].astype(str)
# all_article_df.to_excel('data/article/A2_article_with_sentences_split_four_class_1109.xlsx', index=False)

Index(['TextID', 'Title', 'Crisis_Level'], dtype='object')
[738, 312, 118, 72]
Index(['無標註', '自殺與憂鬱', '自殺行為', '其他類型', 'label', 'Crisis_Level', 'TextID'], dtype='object')


In [ ]:
# randomly choose 25 articles from each class
seed = 1
test_df_list = []
train_df_list = []
to_test = [9, 8, 8, 25]
for i in range(4):
    test_df_list.append(all_article_df[all_article_df['Crisis_Level'] == i].sample(n=to_test[i], random_state=seed))
    train_df_list.append(all_article_df[all_article_df['Crisis_Level'] == i].drop(test_df_list[i].index))
test_df = pd.concat(test_df_list, axis=0, ignore_index=True)
train_df = pd.concat(train_df_list, axis=0, ignore_index=True)

# save
# test_df.to_excel('data/article/A2_final_test_100_1112.xlsx', index=False)
# train_df.to_excel('data/article/A2_final_wo_test_100_1112.xlsx', index=False)
test_df.to_excel('data/article/A2_final_test_50_1113_v2.xlsx', index=False)
train_df.to_excel('data/article/A2_final_wo_test_50_1113_v2.xlsx', index=False)

In [4]:
none_crisis_df = all_article_df.loc[all_article_df['Crisis_Level'] != 3]
partial_none_crisis_df = none_crisis_df.sample(n=72, random_state=1)

In [5]:
crisis_df = all_article_df.loc[all_article_df['Crisis_Level'] == 3]

In [6]:
target_df = pd.concat([crisis_df, partial_none_crisis_df], axis=0, ignore_index=True)
stat = [0]*4
print(target_df)
for idx, val in enumerate(target_df['Crisis_Level']):
    # label = 3 - ord(val[0]) + ord('a')
    stat[val] += 1
print(stat)

                                                   無標註  \
0     呃，這是我跟我媽多年來的秘密 其實也不知道算不算秘密， 但就是不能被我同母異父的弟妹知道的...   
1     要畢業了  每一個人都比我厲害 另一半都很愛她們 也完全不用擔心沒有工作 家人跟朋友愛我 ...   
2     文章很長 畢竟也是超過十年的心路歷程 更深一步的原因要回溯到更小的時候 我永遠記得我媽像發...   
3     在我國小的時候我爸媽就離婚了 我媽帶著我搬出來 她的辛苦 除了她自己我看得比誰都多比誰都了...   
4     我也不知道會不會有人看，不過沒關係。哈哈 於是我就開始裝的我很快樂， 我就搞笑給身邊的所有...   
..                                                 ...   
139             而且說話都很不客氣 一點禮貌也沒有 要怎麼脫離這種情況 說突然不理好像怪怪的   
140   身在國外， 由於中國人在國外佔大多數且對於台獨主義者不友善， 去了Met博物館走走， 在外...   
141   我們家，每年過年，都不是很平靜... 至於今年, 由於先前 我媽一直要我 教我姐姐包裝乾燥...   
142   常常跟我媽吵架，但在外人眼中很奇怪的我們仍然睡在一起。 吵架後在睡前看著我媽的睡臉嫌棄， ...   
143   妳總是反覆的給我希望 總有一天會把力氣用光 我會覺得很可惜 在這種時候又遇見 每次都是這樣...   

                                                 自殺與憂鬱  \
0     我的存在是捏造的謊言。 因此我長大到現在，最恨的就是別人巴我頭、把我的頭或是臉壓下去。 我...   
1     沒有厲害的技能 大概最多只能當助理 領著微薄的薪水苟活一輩子 我不笨 卻又不夠聰明 沒有聰...   
2     懷疑自己有憂鬱症很久了 就像自虐一樣， 想家人罵我的時候、想別人的眼光、甚至用不堪詞語罵自...   
3     但她最後都拿我懷胎十個月很辛苦、我拉拔你長大不容易或我做什麼 說我不接電話、不接沒關係、不...   
4     因為在現實沒有

In [14]:
target_df.to_excel('data/article/A2_article_with_sentences_split_four_class_partial.xlsx', index=False)

In [16]:
test_40_path = 'data/article/A2_final_test_40.xlsx'
test_40_df = pd.read_excel(test_40_path)

# retrieve TextID
test_40_textID = test_40_df['TextID'].values
test_40_textID = [str(val) for val in test_40_textID]
all_article_df['TextID'] = all_article_df['TextID'].astype(str)

# select test_40_textID from all_article_df
test_40_article_df = all_article_df.loc[all_article_df['TextID'].isin(test_40_textID)]
# remove test_40_textID from all_article_df
test_wo_40_article_df = all_article_df.loc[~all_article_df['TextID'].isin(test_40_textID)]


In [17]:
test_40_article_df.to_excel('data/article/A2_final_test_40_1108.xlsx', index=False)
test_wo_40_article_df.to_excel('data/article/A2_final_wo_test_40_1108.xlsx', index=False)

In [ ]:
import torch
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np
label_path = 'data/article/A2_sentence_labeled_v2.xlsx'
only_article_path = 'data/article/A2_二次標註_網路危機訊息標註_1100927_僅全文標註(給致寧).xlsx'
# label_path = 'data/article/partial_A1_article.xlsx'
df = pd.read_excel(label_path)
article_only_df = pd.read_excel(only_article_path)
df = df[['TextID','Title','Crisis_Level']]
print(df.columns)
# print(df)

for idx, val in enumerate(df['TextID']):
    if type(val) != str:
        df['TextID'][idx] = str(val)
stat = [0]*4
for idx, val in enumerate(article_only_df['Crisis_Level']):
    label = 3 - ord(val[0]) + ord('a')
    stat[label] += 1
print(stat)
four_label = df.values
title_list = df['Title'].values
textID_list = df['TextID'].values
sentence_path = 'data/article/A2_sentence_labeled_v2.xlsx'
s_df = pd.read_excel(sentence_path)[[ 'TextID','Title', 'Sentence', '無標註', '自殺與憂鬱',
       '無助或無望', '正向文字', '其他負向文字', '生理反應或醫療狀況', '自殺行為']]
for idx, val in enumerate(s_df['TextID']):
    if type(val) != str:
        s_df['TextID'][idx] = str(val)

sentence_label_list = ['無標註', '自殺與憂鬱', '無助或無望', '正向文字', '其他負向文字',
       '生理反應或醫療狀況', '自殺行為']
article_dict = {str(key): {'無標註':'', '自殺與憂鬱':'',
       '無助或無望':'', '正向文字':'', '其他負向文字':'', '生理反應或醫療狀況':'', '自殺行為':''} for key in textID_list}
problem_title = []
for index, row in s_df.iterrows():
    article_dict[str(row['TextID'])]['label'] = df[(df['TextID']) == (row['TextID'])]['Crisis_Level']
    flag = 3 - ord(article_dict[str(row['TextID'])]['label'].values[0][0]) + ord('a')
    # if flag == 3:
    #    flag = 1
    # elif flag == 1:
    #    flag = 3
    article_dict[str(row['TextID'])]['Crisis_Level'] = flag
    article_dict[str(row['TextID'])]['label'] = [0.0, 0.0, 0.0, 0.0]
    article_dict[str(row['TextID'])]['label'][flag] += 1.0
    for s_label in sentence_label_list:
      if row[s_label] == 1:
        tar = '_' + str(row['Sentence'])
        article_dict[str(row['TextID'])][s_label] += tar
        break
all_article = []
for dic in article_dict:
    single_article = pd.DataFrame([article_dict[dic]])
    single_article['TextID'] = dic
    all_article.append(single_article)
all_article_df = pd.concat(all_article,axis=0, ignore_index=True)
print(all_article_df.columns)
all_article_df['TextID'] = all_article_df['TextID'].astype(str)
all_article_df.to_excel('data/article/A2_article_with_sentences_split.xlsx', index=False)